# Templates, chains and multi-chaining

This example cover a few different topics:
- How to use LangChain Expression Language (LCEL) to build simple text generation chains.
- Create prompt templates, connects them to an OpenAI chat model (gpt-5-nano), and parse the model’s output into plain text.
- Use multi-chaining where one chains output is fed into another chain. In this notebook, a function is generated from the first chain and the second chain generates its test case.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
llm = ChatOpenAI(model_name="gpt-5-nano")

In [7]:
code_prompt = ChatPromptTemplate.from_template(
    "Write a very short {language} function that will {task}"
)

In [10]:
# This is LCEL (LangChain Expression Language). Langchains new method of chaining.
chain = code_prompt | llm | StrOutputParser()

In [11]:
result = chain.invoke({
    "language": "python",
    "task": "return a list of numbers"
})

In [ ]:
print(result)

def numbers(n): return list(range(n))


## Multi-chaining

In [15]:
code_prompt = ChatPromptTemplate.from_template(
    "Write a very short {language} function that will {task}."
)

test_prompt = ChatPromptTemplate.from_template(
    "Write test code for the following {language} code:\n{code}"
)

In [16]:
code_chain = code_prompt | llm | StrOutputParser()
test_chain = test_prompt | llm | StrOutputParser()

In [20]:
code_result = code_chain.invoke({"task": "return a list of numbers", "language": "Python"})
test_result = test_chain.invoke({"language": "Python", "code": code_result})

In [ ]:
print(">>>>>> GENERATED CODE:")
print(code_result)
print(">>>>>> GENERATED TEST:")
print(test_result)

>>>>>> GENERATED CODE:
def nums():
    return [1, 2, 3, 4, 5]
>>>>>> GENERATED TEST:
Here are two common test approaches. Replace your_module with the actual module name that defines nums.

Option 1: pytest

# tests/test_nums.py
from your_module import nums  # replace with the actual module name

def test_nums_returns_expected_list():
    assert nums() == [1, 2, 3, 4, 5]

def test_nums_list_integrity():
    res = nums()
    assert isinstance(res, list)
    assert len(res) == 5
    assert all(isinstance(x, int) for x in res)

def test_nums_new_list_each_call():
    a = nums()
    b = nums()
    assert a == b
    assert a is not b

Option 2: unittest

# tests/test_nums.py
import unittest
from your_module import nums  # replace with the actual module name

class TestNums(unittest.TestCase):
    def test_basic(self):
        self.assertEqual(nums(), [1, 2, 3, 4, 5])

    def test_types_and_length(self):
        res = nums()
        self.assertIsInstance(res, list)
        self.assertEqual(

# Older way of doing chaining

In [ ]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
api_key = ""

llm = OpenAI(
    openai_api_key=api_key
)

In [ ]:
code_prompt = PromptTemplate(
    template="Write a very short {language} function that will {task}",
    input_variables=["language", "task"]
)

In [ ]:
code_chain = LLMChain(
    llm=llm,
    prompt=code_prompt
)

In [ ]:
result = code_chain({
    "language": "python",
    "task": "return a list of numbers"
})

In [ ]:
print(result["text"])

## Multi Chaining

In [ ]:
from langchain.chains import LLMChain, SequentialChain

In [ ]:
code_prompt = PromptTemplate(
    input_variables=["task", "language"],
    template="Write a very short {language} function that will {task}."
)
test_prompt = PromptTemplate(
    input_variables=["language", "code"],
    template="Write test code for the following {language} code:\n{code}"
)

In [ ]:
code_chain = LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code"
)
test_chain = LLMChain(
    llm=llm,
    prompt=test_prompt,
    output_key="test"
)


In [ ]:
chain = SequentialChain(
    chains=[code_chain, test_chain],
    input_variables=["task", "language"],
    output_variables=["test", "code"]
)
result = chain({
    "language": "Python",
    "task": "return a list of numbers"
})

In [ ]:
print(">>>>>> GENERATED CODE:")
print(result["code"])
print(">>>>>> GENERATED TEST:")
print(result["test"])